The first task in recreating the DISA database was migrating data from the existing Mongo db. First, let's take a look at a sample record from the Mongo db (**disamg**), the contents of which have been dumped into a file *latest_entries.json*. 

In [1]:
import json
from collections import namedtuple
import datetime as dt
import pandas as pd

with open('latest-entries.json') as f:
    data = json.load(f)

In [2]:
data[0]

{'_id': {'$oid': '598b4c8baecb6c0001a29e25'},
 'person': {'names': [{'firstName': 'Elizabeth',
    'lastName': '',
    'type': 'English'}],
  'tribe': 'Unspecified',
  'origin': '',
  'race': 'Indian',
  'sex': 'Female',
  'typeKindOfEnslavement': 'Servant',
  'vocation': '',
  'father': {'name': {'firstName': '', 'lastName': ''},
   'race': '',
   'origin': '',
   'status': '',
   'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
    'vocation': ''}},
  'mother': {'name': {'firstName': '', 'lastName': ''},
   'race': '',
   'origin': '',
   'status': '',
   'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
    'vocation': ''}},
  'children': []},
 'owner': {'name': {'firstName': '', 'lastName': 'Duglas', 'title': 'Mr. '},
  'vocation': ''},
 'dateOfRunaway': {'month': ''},
 'dateOfEmancipation': {'month': ''},
 'buyer': {'firstName': '', 'lastName': ''},
 'seller': {'firstName': '', 'lastName': ''},
 'dateOfSale': {'month': ''},
 'dateOfMarriage': {'mon

We see a number of potential issues here. Let's consider each in turn.

### Nested Data

Each entry in any Mongo db is a JSON object, which itself can contain an arbitrary number of JSON objects. Here are the top-level attributes for all entries in **disamg**:
There are several nested top-level objects: `_id`, `document`, `meta`, `person`, `owner`, `buyer`, `seller`, 

In [16]:
all_attrs = { k for d in data for k in d.keys() }
for a in sorted(list(all_attrs)):
    print(a)

_id
additionalInformation
buyer
dateOfDeath
dateOfEmancipation
dateOfMarriage
dateOfRunaway
dateOfSale
document
meta
owner
person
researcherNotes
seller


This is an odd assortment of labels; it's not immediately clear what each entry in **disamg** represents. Considering the context of the DISA project, the guess is that `person` is the central data point, with `buyer`, `seller`, `owner`, and the 5 `dateOf*` fields referring back to `person` (ie, 'owner of `person`, 'date of death of `person`, etc.). `document` presumably identifies the document where the `person` appears, and `_id`, `meta`, `additionalInformation`, and `researcherNotes` all containing some administrative-level information.

The attributes `researcherNotes` and `additionalInformation` are both text strings; all the rest of the attributes are themselves JSON objects containing more data. The `_id`, `owner`, `buyer`, `seller`, and `dateOf*` attributes are simple objects,

![title](img/DISA_er_1.png)

In [22]:
FlatRow = namedtuple('FlatRow', [
    #metadata
    'mongo_id', 'disa_id', 'modified_date', 'creator', 'last_updated_by',
    #name information
    'first_name', 'last_name', 'name_count',
    #personal details
    'tribe', 'origin', 'race', 'sex', 'age', 'enslavement', 'vocation',
    #personal event dates
    'date_runaway', 'date_sale', 'date_emancipation', 'date_marriage', 'date_death',
    #owner information
    'owner_first', 'owner_last', 'owner_title', 'owner_vocation',
    'buyer_first', 'buyer_last', 'seller_first', 'seller_last',
    #father information
    'father_first', 'father_last', 'father_origin', 'father_race', 'father_status',
    #father owner information
    'father_owner_first', 'father_owner_last', 'father_owner_title','father_owner_vocation',
    #mother information
    'mother_first', 'mother_last', 'mother_origin', 'mother_race', 'mother_status',
    #mother owner information
    'mother_owner_first', 'mother_owner_last', 'mother_owner_title', 'mother_owner_vocation',
    #child information
    'child_first', 'child_last', 'child_count',
    #location information
    'nation', 'state', 'location', 'locale',
    #document information
    'document_date', 'source', 'record', 'citation',
    #text blobs
    'addl_info', 'notes' ])

In [69]:
def get_date(dateData):
    if not dateData or dateData == {'month': ''} or dateData == {'year': '', 'month': '', 'day':''}:
        dateData = { 'day':1, 'month':1, 'year':1 }
    try:
        day = int( dateData.get('day',1) or 1)
        month = int( dateData.get('month',1) or 1)
        year = int( dateData.get('year',1) or 1)
    except:
        day = 1
        month = 1
        year = 1
    return dt.datetime(day=day, month=month, year=year) 

In [80]:
def nest(jd, *args):
    nest = jd
    for arg in args:
        try:
            nest = nest[arg]
        except:
            return ''
    return nest

def flatten(jd):
    row = FlatRow(
        mongo_id = nest(jd, '_id', '$oid'),
        disa_id = '{}-{}'.format(nest(jd,'meta','idPrefix'),nest(jd,'meta','idSuffix')),
        modified_date = dt.datetime.strptime(
            nest(jd, 'meta','lastModified'), '%Y-%m-%dT%H:%M:%S.%fZ'),
        creator = nest(jd, 'meta', 'creator'),
        last_updated_by = nest(jd, 'meta', 'updatedBy'),
        first_name = nest(jd, 'person', 'names', 0, 'firstName'),
        last_name = nest(jd, 'person', 'names', 0, 'firstName'),
        name_count = len( nest(jd, 'person', 'names') ),
        tribe = nest(jd, 'person', 'tribe'),
        origin = nest(jd, 'person', 'origin'),
        race = nest(jd, 'person', 'race'),
        sex = nest(jd, 'person', 'sex'),
        age = nest(jd, 'person', 'age'),
        enslavement = nest(jd, 'person', 'typeKindOfEnslavement'),
        vocation = nest(jd, 'person', 'vocation'),
        date_runaway = get_date( nest(jd, 'dateOfRunaway') ),
        date_sale = get_date( nest(jd, 'dateOfSale') ),
        date_emancipation = get_date( nest(jd, 'dateOfEmancipation') ),
        date_marriage = get_date( nest(jd, 'dateOfMarriage') ),
        date_death = get_date( nest(jd, 'dateOfDeath') ),
        owner_first = nest(jd, 'owner', 'firstName'),
        owner_last = nest(jd, 'owner', 'lastName'),
        owner_title = nest(jd, 'owner', 'title'),
        owner_vocation = nest(jd, 'owner', 'vocation'),
        buyer_first = nest(jd, 'buyer', 'firstName'),
        buyer_last = nest(jd, 'buyer', 'lastName'),
        seller_first = nest(jd, 'seller', 'firstName'),
        seller_last = nest(jd, 'seller', 'firstName'),
        father_first = nest(jd, 'person', 'father', 'name', 'firstName'),
        father_last = nest(jd, 'person', 'father', 'name', 'lastName'),
        father_origin = nest(jd, 'person', 'father', 'origin'),
        father_race = nest(jd, 'person', 'father', 'race'),
        father_status = nest(jd, 'person', 'father', 'status'),
        father_owner_first = nest(jd, 'person', 'father', 'owner', 'name', 'firstName'),
        father_owner_last = nest(jd, 'person', 'father', 'owner', 'name', 'lastName'),
        father_owner_title = nest(jd, 'person', 'father', 'owner', 'title'),
        father_owner_vocation = nest(jd, 'person', 'father', 'owner', 'vocation'),
        mother_first = nest(jd, 'person', 'mother', 'name', 'firstName'),
        mother_last = nest(jd, 'person', 'mother', 'name', 'lastName'),
        mother_origin = nest(jd, 'person', 'mother', 'origin'),
        mother_race = nest(jd, 'person', 'mother', 'race'),
        mother_status = nest(jd, 'person', 'mother', 'status'),
        mother_owner_first = nest(jd, 'person', 'mother', 'owner', 'name', 'firstName'),
        mother_owner_last = nest(jd, 'person', 'mother', 'owner', 'name', 'lastName'),
        mother_owner_title = nest(jd, 'person', 'mother', 'owner', 'title'),
        mother_owner_vocation = nest(jd, 'person', 'mother', 'owner', 'vocation'),
        child_first = nest(jd, 'person', 'children', 0, 'name', 'firstName'),
        child_last = nest(jd, 'person', 'children', 0, 'name', 'lastName'),
        child_count = len( nest(jd, 'person', 'children') ) ,
        nation = nest(jd, 'document', 'nationalContext'),
        state = nest(jd, 'document', 'colonyState'),
        location = nest(jd, 'document', 'stringLocation'),
        locale = nest(jd, 'document', 'locale'),
        document_date = get_date( nest(jd, 'document', 'date') ),
        source = nest(jd, 'document', 'sourceType'),
        record = nest(jd, 'document', 'recordType'),
        citation = nest(jd, 'document', 'citation'),
        addl_info = nest(jd, 'additionalInformation'),
        notes = nest(jd, 'researcherNotes') )

    return row

In [81]:
flat = [ flatten(rec) for rec in data ]
flat[0]

FlatRow(mongo_id='598b4c8baecb6c0001a29e25', disa_id='10-10002', modified_date=datetime.datetime(2018, 5, 14, 18, 32, 21, 539000), creator='112148132795694739523', last_updated_by='112487255676465508755', first_name='Elizabeth', last_name='Elizabeth', name_count=1, tribe='Unspecified', origin='', race='Indian', sex='Female', age='', enslavement='Servant', vocation='', date_runaway=datetime.datetime(1, 1, 1, 0, 0), date_sale=datetime.datetime(1, 1, 1, 0, 0), date_emancipation=datetime.datetime(1, 1, 1, 0, 0), date_marriage=datetime.datetime(1, 1, 1, 0, 0), date_death=datetime.datetime(1, 1, 1, 0, 0), owner_first='', owner_last='', owner_title='', owner_vocation='', buyer_first='', buyer_last='', seller_first='', seller_last='', father_first='', father_last='', father_origin='', father_race='', father_status='', father_owner_first='', father_owner_last='', father_owner_title='', father_owner_vocation='', mother_first='', mother_last='', mother_origin='', mother_race='', mother_status='',

In [82]:
disa = pd.DataFrame(flat)

In [83]:
disa

,mongo_id,disa_id,modified_date,creator,last_updated_by,first_name,last_name,name_count,tribe,origin,...,nation,state,location,locale,document_date,source,record,citation,addl_info,notes
0,598b4c8baecb6c0001a29e25,10-10002,2018-05-14 18:32:21.539,112148132795694739523,112487255676465508755,Elizabeth,Elizabeth,1,Unspecified,,...,British,Connecticut,New London,,1671-05-07 00:00:00,Book,,"Blake, The Later History of the First Church o...",,
1,599c82feaecb6c0001a29e26,10-10003,2018-06-04 21:26:02.976,112148132795694739523,103795391716629952261,Harry,Harry,1,Unspecified,,...,British,Narragansett (Rhode Island),Kingstown,,1704-06-12 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00004. America's Historical New...,"Details of advertisement: ""Ran away from Capt....",Although his owner is the aforementioned Natha...
2,599c8586aecb6c0001a29e27,10-10004,2018-05-22 22:58:51.373,112148132795694739523,109503335312366098524,Prince,Prince,1,Unspecified,,...,British,Massachusetts,Boston,,1704-11-06 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00006; America's Historical New...,"Details of advertisement: ""Ran-away on Wednesd...",
3,599c8922aecb6c0001a29e28,10-10005,2018-06-04 21:39:13.923,112148132795694739523,103795391716629952261,Hannah,Hannah,1,,,...,British,Massachusetts,Barnstable,,1706-02-17 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00012; America's Historical New...,"Details of advertisement: ""Ran-away the last S...",
4,599c8b4faecb6c0001a29e29,10-10006,2018-06-04 21:35:30.437,112148132795694739523,103795391716629952261,Grace,Grace,1,,,...,British,New York,,,1706-01-13 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00010; America's Historical New...,"Details of advertisement: ""Ran-away from her M...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,5b0498c0de60e011b9da3802,10-10490,2018-05-23 04:57:46.281,109503335312366098524,109503335312366098524,Milly,Milly,1,,,...,,Connecticut,Stonington,,1716-08-20 00:00:00,Newspaper,Runaway Advertisement,Zotero ID: DISA00070; America's Historical New...,"Details of advertisement: ""Ran away from Mr. D...",
489,5b0499bbde60e00a7cda3803,10-10491,2018-05-23 04:58:07.227,109503335312366098524,109503335312366098524,,,0,,,...,,Massachusetts,Boston,,1716-08-27 00:00:00,Newspaper,Advertisement of sale,Zotero ID: DISA00071; America's Historical New...,"Details of sale: ""A Spanish Indian Man Servant...",
490,5b049b52de60e03fe7da3804,10-10492,2018-05-22 22:37:09.231,109503335312366098524,109503335312366098524,,,1,,,...,,,,,0001-01-01 00:00:00,,,,,
491,5b049c9fde60e02bdada3805,10-10493,2018-05-23 17:23:40.210,109503335312366098524,112148132795694739523,,,0,,,...,British,Massachusetts,Boston,Sudbury Street,1716-10-01 00:00:00,Newspaper,Advertisement of sale,Zotero ID: DISA00073; America's Historical New...,"Details of sale: ""An Indian woman of about Twe...",


In [84]:
disa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 59 columns):
mongo_id                 493 non-null object
disa_id                  493 non-null object
modified_date            493 non-null datetime64[ns]
creator                  493 non-null object
last_updated_by          493 non-null object
first_name               493 non-null object
last_name                493 non-null object
name_count               493 non-null int64
tribe                    493 non-null object
origin                   493 non-null object
race                     493 non-null object
sex                      493 non-null object
age                      493 non-null object
enslavement              493 non-null object
vocation                 493 non-null object
date_runaway             493 non-null object
date_sale                493 non-null object
date_emancipation        493 non-null object
date_marriage            493 non-null object
date_death               493 non-

In [86]:
disa[ disa.child_count != 0 ]

,mongo_id,disa_id,modified_date,creator,last_updated_by,first_name,last_name,name_count,tribe,origin,...,nation,state,location,locale,document_date,source,record,citation,addl_info,notes
11,599cab56aecb6c0001a29e30,10-10013,2017-12-29 19:27:54.141,113112063790792171857,112148132795694739523,Rachel,Rachel,1,Mohegan,,...,British,Connecticut,New London,,1720-12-04 00:00:00,Archive,,"Connecticut Church Records, Index, New London ...",,
22,599f04bdaecb6c0001a29e3b,10-10024,2017-12-29 19:36:48.169,113112063790792171857,112148132795694739523,John,John,1,Unspecified,,...,British,Jamaica,Port Royal,,1749-04-27 00:00:00,Archive,Manumission,"Jamaica Archives, Manumission of Slaves; “Manu...",,Same name between child and slave.
91,59ea38c1df4c6f000198334a,10-10093,2018-05-31 16:32:56.581,112148132795694739523,113112063790792171857,Joan,Joan,1,,Pueblo de Pauto,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",,
92,5a0b84b6df4c6f000198334b,10-10094,2018-05-31 16:33:08.834,112148132795694739523,113112063790792171857,Catalina,Catalina,1,,Pueblo de Casanare,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",Married to Joan Pauto,Husband's ID in database: DISA-10-10093
119,5a3152addf4c6f0001983366,10-10121,2018-06-15 15:06:14.275,112148132795694739523,113112063790792171857,Joan,Joan,1,,,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",,Son's ID in database: DISA-10-10123
120,5a316c27df4c6f0001983367,10-10122,2018-06-15 15:06:41.665,112148132795694739523,113112063790792171857,Joan,Joan,1,,Casanave,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",Joan is noted as a widower (viudo),


In [87]:
disa.iloc[92].date_marriage

datetime.datetime(1, 1, 1, 0, 0)

In [90]:
disa[ disa.addl_info.str.contains("owner") ]

,mongo_id,disa_id,modified_date,creator,last_updated_by,first_name,last_name,name_count,tribe,origin,...,nation,state,location,locale,document_date,source,record,citation,addl_info,notes
55,59c26e8edf4c6f0001983326,10-10057,2018-06-14 01:13:54.377,113112063790792171857,103795391716629952261,Israel,Israel,1,Unspecified,"Allentown, PA",...,British,Pennsylvania,Philadelphia,,1791-10-25 00:00:00,Newspaper,Runaway advertisement,DISA00094; America's Historical Newspapers [Ma...,Details of advertisement: “New Books. Just rec...,The same advertisement also appears in the Mar...
126,5a383cc8df4c6f000198336d,10-10128,2018-06-15 15:18:44.544,113112063790792171857,113112063790792171857,Pickle,Pickle,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-02-24 00:00:00,Archival,Registry,DISA00123; “Return of the Registry of Indians ...,"These records are very sparse. Same owner as ""...",
127,5a383e8ddf4c6f000198336e,10-10129,2018-06-15 15:18:58.994,113112063790792171857,113112063790792171857,Johnny,Johnny,1,Unspecified,,...,British,British Honduras,Mosquito Shore,,1777-02-24 00:00:00,Archive,Registry,DISA00123; “Return of the Registry of Indians ...,Same owner as that of Pickle,
129,5a3841bfdf4c6f0001983370,10-10131,2018-06-15 15:19:56.478,113112063790792171857,113112063790792171857,Jacob,Jacob,1,Unspecified,,...,British,British Honduras,Mosquito Shore,,1777-02-24 00:00:00,Archive,Registry,DISA00123; “Return of the Registry of Indians ...,Same owner as Catto.,
130,5a384276df4c6f0001983371,10-10132,2018-06-15 15:20:09.808,113112063790792171857,113112063790792171857,Catto,Catto,1,Unspecified,,...,British,British Honduras,Mosquito Shore,,1777-02-24 00:00:00,Archive,Registry,DISA00123; “Return of the Registry of Indians ...,Same owner as Jacob,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,5a949c181b47910001a53b2d,10-10431,2018-06-27 00:54:23.456,113112063790792171857,112148132795694739523,Mystilla,Mystilla,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",
430,5a949d191b47910001a53b2e,10-10432,2018-06-27 00:54:44.375,113112063790792171857,112148132795694739523,Susannah,Susannah,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",
431,5a949dca1b47910001a53b2f,10-10433,2018-06-27 00:54:53.798,113112063790792171857,112148132795694739523,Quash,Quash,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",
432,5a949e171b47910001a53b30,10-10434,2018-06-27 00:55:49.830,113112063790792171857,112148132795694739523,Docinda,Docinda,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",


In [91]:
disa.disa_id.values

array(['10-10002', '10-10003', '10-10004', '10-10005', '10-10006',
       '10-10007', '10-10008', '10-10009', '10-10010', '10-10011',
       '10-10012', '10-10013', '10-10014', '10-10015', '10-10016',
       '10-10017', '10-10018', '10-10019', '10-10020', '10-10021',
       '10-10022', '10-10023', '10-10024', '10-10025', '10-10026',
       '10-10027', '10-10028', '10-10029', '10-10030', '10-10031',
       '10-10032', '10-10033', '10-10034', '10-10035', '10-10036',
       '10-10037', '10-10038', '10-10039', '10-10040', '10-10041',
       '10-10042', '10-10043', '10-10044', '10-10045', '10-10046',
       '10-10047', '10-10048', '10-10049', '10-10050', '10-10051',
       '10-10052', '10-10053', '10-10054', '10-10055', '10-10056',
       '10-10057', '10-10058', '10-10059', '10-10060', '10-10061',
       '10-10062', '10-10063', '10-10064', '10-10065', '10-10066',
       '10-10067', '10-10068', '10-10069', '10-10070', '10-10071',
       '10-10072', '10-10073', '10-10074', '10-10075', '10-100

In [94]:
disa.nation.value_counts()

British          365
                  54
Spanish           28
British           22
American          12
USA                6
British Coast      2
Honduras           2
United States      1
French             1
Name: nation, dtype: int64